# Классификация интентов
В этом ноутбуке произведем классификацию интентов: **повар** (https://www.kaggle.com/datasets/coolonce/recipes-and-interpretation-dim), **врач** (blinoff/medical_qa_ru_data), **болталка** (https://www.kaggle.com/datasets/mrapplexz/bashim-quotes).

Скачаем данные и установим необходимое окружение:

In [12]:
! wget https://huggingface.co/datasets/blinoff/medical_qa_ru_data/resolve/main/medical_qa_ru_data.csv

! pip install corus

--2023-09-19 04:44:15--  https://huggingface.co/datasets/blinoff/medical_qa_ru_data/resolve/main/medical_qa_ru_data.csv
Resolving huggingface.co (huggingface.co)... 99.84.160.64, 99.84.160.9, 99.84.160.43, ...
Connecting to huggingface.co (huggingface.co)|99.84.160.64|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/datasets/blinoff/medical_qa_ru_data/6aca4fb6dae499745c9ca1dfad8a0ade809573daf228c923fdf846ba4a639b50?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27medical_qa_ru_data.csv%3B+filename%3D%22medical_qa_ru_data.csv%22%3B&response-content-type=text%2Fcsv&Expires=1695357855&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5NTM1Nzg1NX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9kYXRhc2V0cy9ibGlub2ZmL21lZGljYWxfcWFfcnVfZGF0YS82YWNhNGZiNmRhZTQ5OTc0NWM5Y2ExZGZhZDhhMGFkZTgwOTU3M2RhZjIyOGM5MjNmZGY4NDZiYTRhNjM5YjUwP3Jlc3BvbnNlLWNvbnRlbnQtZGlzcG9zaXRpb2

In [13]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.5 MB/s eta 0:00:00


Примонтируем гугл-диск и скачаем файл с вопросами и ответами мейла:

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [7]:
!ls "/content/drive/MyDrive/Data science_data/nlp/project/"

all_recepies_inter.csv	bash_set.jsonl


In [5]:
!ls "/content/drive/MyDrive/Data Science_models/nlp/project/"

model_qa_medical_gen  tokenizer_qa_medical_gen
model_qa_рецепт_gen   tokenizer_qa_рецепт_gen


In [8]:
path_d =  "/content/drive/MyDrive/Data science_data/nlp/project/"

In [9]:
path_m =  "/content/drive/MyDrive/Data Science_models/nlp/project/"

In [14]:
import numpy as np
import pandas as pd
import json
import re
from corus import load_rudrec
from tqdm import tqdm_notebook
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, classification_report

In [15]:
df_rec = pd.read_csv(path_d + 'all_recepies_inter.csv', sep='\t')
df_rec

,Unnamed: 0,name,composition,cooking_type,Инструкции,dish_type,Дата,photo,source,composition_inter
0,0,рассольник классический с перловкой и солеными...,"[{'Перловка': 0.1, 'unit': 'стак. (200 мл)'}, ...","варка,жарка",Подготовить указанные ингредиенты для приготов...,первое,05.06.2015,photo_1000menu_1.jpg,https://1000.menu/cooking/33395-rassolnik-s-pe...,"[{'product_id': 4253, 'name_source': 'Перловая..."
1,1,Суп пюре из белокочаной капусты,"[{'Капуста белокочанная': 50.0, 'unit': 'гр'},...",варка,"Необходимые ингредиенты\r\nНарезаем лук, морко...",первое,27.06.2015,photo_1000menu_2.jpg,https://1000.menu/cooking/25399-sup-pure-iz-be...,"[{'product_id': 2286, 'name_source': 'Капуста ..."
2,2,Постные щи из квашеной капусты,"[{'Капуста квашеная': 116.7, 'unit': 'гр'}, {'...","варка,жарка,тушение","Честно признаюсь, у меня не было репы на момен...",первое,12.02.2013,photo_1000menu_3.jpg,https://1000.menu/cooking/5159-postnje-shchi,"[{'product_id': 0, 'name_source': 'Капуста ква..."
3,3,Тюря- простой суп быстро и вкусно,"[{'Квас': 0.2, 'unit': 'л'}, {'Лук репчатый': ...",сырое,"\r\nНачинаем мы приготовление тюри с того, что...",первое,02.03.2011,photo_1000menu_4.jpg,https://1000.menu/cooking/5085-turya,"[{'product_id': 0, 'name_source': 'Квас', 'uni..."
4,4,Фасолевый суп из красной фасоли,"[{'Вода': 0.3, 'unit': 'л'}, {'Картошка': 0.3,...",варка,Подготовить ингредиенты. Для приготовления суп...,первое,28.01.2013,photo_1000menu_5.jpg,https://1000.menu/cooking/38765-fasolevyi-sup-...,"[{'product_id': 828, 'name_source': 'Вода', 'u..."
...,...,...,...,...,...,...,...,...,...,...
27879,26468,Салат с филе тунца и огурцами в азиатской…,"[{'Тунец': 50.0, 'unit': 'г'}, {'Огурцы': 50.0...",жарка,1. Для приготовления азиатского соуса смешать ...,салат,20.05.2015,photo_eda_28463.jpg,https://eda.ru/recepty/salaty/salat-s-file-tun...,"[{'product_id': 5989, 'name_source': 'Тунец', ..."
27880,26469,Теплый салат с куриной печенью и яблоками,"[{'Куриная печень': 120.0, 'unit': 'г'}, {'Кис...","варка,жарка,запекание","1. Маленькими ломтиками порезать яблоки, запек...",салат,27.09.2018,photo_eda_28464.jpg,https://eda.ru/recepty/salaty/teplyy-salat-s-k...,"[{'product_id': 0, 'name_source': 'Куриная печ..."
27881,26470,Самый зеленый салат,"[{'Мангольд': 'по вкусу', 'unit': 'по вкусу'},...",сырое,"1. Чеснок натереть, сыр фета раскрошить.\r\n2....",салат,24.03.2018,photo_eda_28466.jpg,https://eda.ru/recepty/salaty/samyy-zelenyy-sa...,"[{'product_id': 3378, 'name_source': 'Мангольд..."
27882,26471,Теплый салат с тыквой и брынзой под…,"[{'Тыква': 100.0, 'unit': 'г'}, {'Сыр брынза':...",запекание,"1. Тыкву нарезаем кубиками, добавляем оливково...",салат,14.05.2019,photo_eda_28467.jpg,https://eda.ru/recepty/salaty/teplyy-salat-s-t...,"[{'product_id': 6023, 'name_source': 'Тыква', ..."


In [16]:
data_0 = df_rec[['name']]
data_0.dropna(subset=['name'], inplace=True)
data_0['class'] = 0
data_0

<ipython-input-16-91644ec12017>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_0.dropna(subset=['name'], inplace=True)
<ipython-input-16-91644ec12017>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_0['class'] = 0


,name,class
0,рассольник классический с перловкой и солеными...,0
1,Суп пюре из белокочаной капусты,0
2,Постные щи из квашеной капусты,0
3,Тюря- простой суп быстро и вкусно,0
4,Фасолевый суп из красной фасоли,0
...,...,...
27879,Салат с филе тунца и огурцами в азиатской…,0
27880,Теплый салат с куриной печенью и яблоками,0
27881,Самый зеленый салат,0
27882,Теплый салат с тыквой и брынзой под…,0


Посмотрим датасет для второго интента:



In [17]:
data_1 = pd.read_csv('medical_qa_ru_data.csv')
data_1

,date,categ,theme,desc,ans,spec10
0,"8 Октября 2017, 11:55",Оториноларингология,Применение Ларипронта.,"Ларипронт 20 талеток,через каждые 2-3 часа.Оче...",Что вы им лечите? Длительность приема Ларипрон...,Отоларинголог
1,"20 Февраля 2019, 13:24",Акушерство,Беременность,"Здравствуйте, я на 7-8 неделе беременности. С ...","Здравствуйте, это может быть признаком раннего...",NaN
2,"17 Марта 2015, 18:31",Другое,гинекология,Здравствуйте месячные должны придти 23 марта в...,Выполните исследование хгч,NaN
3,"13 Января 2019, 19:38",Терапия,Занятия спорта после сдачи крови,"Завтра иду с утра сдавать кровь ТТГ, Т4СВ, Кал...","Можно.;\nЗдравствуйте , да, попейте сладкого ч...",Терапевт
4,"28 Ноября 2017, 21:58",Другое,Таблетки,Мне прописали пить Аллохол. Врач написала пить...,Препарат принимается после еды. Уточните это ...,NaN
...,...,...,...,...,...,...
190330,"31 Июля 2014, 11:31",Офтальмология,Отлетела в глаз пластмасса,"Здравствуйте! В глаз отлетел кусок пластмассы,...","Ромашка есть дома, альбуцид;\nМожно промыть лю...",Офтальмолог
190331,"16 Марта 2014, 21:52",Другое,здравствуйте скажите пожалуста где я могу посм...,NaN,"В ""консультанте""",NaN
190332,"22 Ноября 2018, 11:02",Общая хирургия,палец,"Здравствуйте, три дня назад порезала палец бу...","Здравствуйте , пока обработать Хлоргексидином ...",Хирург
190333,"2 Ноября 2018, 21:19",Эндокринология,Эндокринолог прописал пить сиофор 500 при саха...,NaN,Такие препараты принимаются постоянно;\nЗдравс...,NaN


Из ответов оставим только первый ответ. Сообщения разделены ';\n'

Из датасета оставим только столбцы 'desc' и 'ans', удалим строки с пустыми значениями, и назначим данному интенту класс 1:

In [18]:
data_1 = data_1[['desc', 'ans']]
data_1.dropna(subset=['desc', 'ans'], inplace=True)
data_1['class'] = 1
data_1

<ipython-input-18-571bca050f5e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_1.dropna(subset=['desc', 'ans'], inplace=True)
<ipython-input-18-571bca050f5e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_1['class'] = 1


,desc,ans,class
0,"Ларипронт 20 талеток,через каждые 2-3 часа.Оче...",Что вы им лечите? Длительность приема Ларипрон...,1
1,"Здравствуйте, я на 7-8 неделе беременности. С ...","Здравствуйте, это может быть признаком раннего...",1
2,Здравствуйте месячные должны придти 23 марта в...,Выполните исследование хгч,1
3,"Завтра иду с утра сдавать кровь ТТГ, Т4СВ, Кал...","Можно.;\nЗдравствуйте , да, попейте сладкого ч...",1
4,Мне прописали пить Аллохол. Врач написала пить...,Препарат принимается после еды. Уточните это ...,1
...,...,...,...
190328,"Не курю и не пью. Веду здоровый образ жизни, м...",В первую очередь вам необходимо обследовать пе...,1
190329,"Здравствуйте, очень прошу консультации. В лево...",Начинать следует с терапевтических методов. Ес...,1
190330,"Здравствуйте! В глаз отлетел кусок пластмассы,...","Ромашка есть дома, альбуцид;\nМожно промыть лю...",1
190332,"Здравствуйте, три дня назад порезала палец бу...","Здравствуйте , пока обработать Хлоргексидином ...",1


Теперь возьмем датасет для 3-го интента - rudrec_annotated. Берем только текст - отзывы на лекарства.

In [8]:
#path = 'bash_set.jsonl'
#records = load_rudrec(path_d)

In [20]:
# Открываем файл и читаем данные
with open(path_d + 'bash_set.jsonl', 'r') as f:
    data = [json.loads(line.strip()) for line in f]

# Создаем пустой словарь для хранения данных
data_2 = {'id': [], 'text': [], 'class': []}

# Обрабатываем каждую запись
for idx, rec in enumerate(data):
    # Получаем текст записи
    text = rec.get('text', '')

    # Получаем класс (если есть) или задаем значение по умолчанию
    classification = rec.get('class', 'unknown')

    # Добавляем данные в словарь
    data_2['id'].append(idx)
    data_2['text'].append(text.strip())
    data_2['class'].append(classification)

# Создаем датафрейм из словаря
data_2 = pd.DataFrame(data_2)
data_2.dropna(subset=['text'], inplace=True)
data_2['class'] = 2

In [21]:
data_2

,id,text,class
0,0,"<Ares> ppdv, все юниксы очень дружелюбны.. они...",2
1,1,<томатик_рад> а ты не чувствуешь красоту мира?...,2
2,2,"<Дор> ""мышка, почему у тебя такие большие глаз...",2
3,3,"<PPDV[os2]> ""Мальчики, вы что больные, бегать ...",2
4,4,<Ohtori_Akio> мы - как разработчики - живём с ...,2
...,...,...,...
81492,81492,"xxx: угадайте не гугля, что такое жопиздан!\ny...",2
81493,81493,"xxx:\nПосетила шальная мысль заняться собой, ж...",2
81494,81494,#всебожьяроса\nxxx: Судьба айтишников вообще н...,2
81495,81495,"Прибывшие на место правоохранители установили,...",2


Все три датасета достаточно объемные для дообучения. Первый датасет (data-0) самый маленький - 27884 строк. Сократим выборки для ускорения обучения, возьмем из всех
датасетов случайным семплированием одинаковое количество строк (10000) и объеденим в один датасет.

In [22]:
data_0 = data_0.sample(10000)
data_0

,name,class
18589,Панакотта,0
25165,Тортильи с ветчиной и сыром,0
11617,Сырники из творога и овсяных хлопьев,0
16352,Маффины с чеддером и паприкой,0
110,Рассольник с фасолью,0
...,...,...
24483,Печеная спаржа с чесночно-лимонным соусом,0
13394,Творожный пирог с цукатами,0
1409,Сыроедческий салат «Февральский»,0
17050,Быстрый шоколадный десерт,0


In [23]:
data_1 = data_1.sample(10000)
data_1

,desc,ans,class
36761,"Здравствуйте. Моему ребёнку 5 лет, вес около 2...",Здравствуйте. Какой пол и рост у ребенка? Род...,1
20385,В горло попал кусочек пуха(от шарфа).И откашля...,"Покажитесь ЛОР врачу.;\nЗдравствуйте, осмотр л...",1
153042,"Добрый вечер, подскажите пожалуйста что это мо...","Здравствуйте. Возможно респираторный синдром, ...",1
83600,"здравствуйте вот уже два года болит голова,сле...",Артериовенозная мальформация — это врожденная ...,1
51084,Скачки давления - от 180/100 до 80/40 за корот...,"Здравствуйте, как давно у вас такое состояние?...",1
...,...,...,...
20291,"В 7 лет поставили специальную пластину, в 14 м...",Нежелательно.;\nпосле контрольных исследований...,1
109878,здравствуйте. я заметила у себя проблему. сухо...,"Непременно провести клинический анализ крови, ...",1
90376,"Мать, 90 лет.\n \n ИБС, ...","Здравствуйте, вам при таком диагнозе должны бы...",1
147992,"Здравствуйте,уже второй раз на груди,ниже соск...","Нужен очный осмотр,по интернету причину не наз...",1


In [24]:
data_2 = data_2.sample(10000)

In [25]:
data_2

,id,text,class
16533,16533,blonde\nпошли погуляем\nfuckmosk\nкста привет ...,2
27793,27793,"xxx: коты едяд либо что-то вкусное, либо что-т...",2
1535,1535,"* Архон наблюдает, как 67-мегабайтный MySQL-да...",2
67316,67316,"yy: Ты знаешь, в каком слипноте это делали?\nt...",2
15623,15623,Вот он - технический прогресс! Когда в ясный д...,2
...,...,...,...
39558,39558,Обсуждение usb-пылесоса в одном из интернет-ма...,2
774,774,* MAD[away_mode] is now known as MAD`wrk\n* MA...,2
72223,72223,<Наталья> Надо внезапным родственникам выдать ...,2
77765,77765,"Ххх: родственники на др задарили пару елочек, ...",2


In [26]:
data_1_tmp = data_1[['desc', 'class']]
data_1_tmp.rename(columns={'desc': 'text'}, inplace=True)
data_1_tmp

<ipython-input-26-2a25633a713c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_1_tmp.rename(columns={'desc': 'text'}, inplace=True)


,text,class
36761,"Здравствуйте. Моему ребёнку 5 лет, вес около 2...",1
20385,В горло попал кусочек пуха(от шарфа).И откашля...,1
153042,"Добрый вечер, подскажите пожалуйста что это мо...",1
83600,"здравствуйте вот уже два года болит голова,сле...",1
51084,Скачки давления - от 180/100 до 80/40 за корот...,1
...,...,...
20291,"В 7 лет поставили специальную пластину, в 14 м...",1
109878,здравствуйте. я заметила у себя проблему. сухо...,1
90376,"Мать, 90 лет.\n \n ИБС, ...",1
147992,"Здравствуйте,уже второй раз на груди,ниже соск...",1


In [27]:
data_0_tmp = data_0[['name', 'class']]
data_0_tmp.rename(columns={'name': 'text'}, inplace=True)
data_0_tmp

,text,class
18589,Панакотта,0
25165,Тортильи с ветчиной и сыром,0
11617,Сырники из творога и овсяных хлопьев,0
16352,Маффины с чеддером и паприкой,0
110,Рассольник с фасолью,0
...,...,...
24483,Печеная спаржа с чесночно-лимонным соусом,0
13394,Творожный пирог с цукатами,0
1409,Сыроедческий салат «Февральский»,0
17050,Быстрый шоколадный десерт,0


In [28]:
data_2 = data_2[['text', 'class']]

In [29]:
data = pd.concat([data_0_tmp, data_1_tmp, data_2], ignore_index=True)
data

,text,class
0,Панакотта,0
1,Тортильи с ветчиной и сыром,0
2,Сырники из творога и овсяных хлопьев,0
3,Маффины с чеддером и паприкой,0
4,Рассольник с фасолью,0
...,...,...
29995,Обсуждение usb-пылесоса в одном из интернет-ма...,2
29996,* MAD[away_mode] is now known as MAD`wrk\n* MA...,2
29997,<Наталья> Надо внезапным родственникам выдать ...,2
29998,"Ххх: родственники на др задарили пару елочек, ...",2


Этот код перетасовывает (случайным образом перемешивает) строки в DataFrame data. Вот, как это происходит:

- data.sample(frac=1) вызывает метод sample() на DataFrame data, который случайным образом выбирает фракцию (долю) строк из DataFrame. Значение frac=1 указывает, что нужно выбрать все строки (полную долю, равную 1).
- reset_index(drop=True) вызывает метод reset_index() на полученном результате, который сбрасывает индексы строк и создает новый индекс, начиная с 0. Параметр drop=True указывает на отбрасывание старого индекса, который не будет сохранен в DataFrame.

In [30]:
data = data.sample(frac=1).reset_index(drop=True)
data

,text,class
0,xxx: Я еще детям своим про крах Мелкософта рас...,2
1,Хумус из свеклы,0
2,Полезная пицца,0
3,"Кальмары, тушенные в сметане",0
4,Нежное песочное печенье,0
...,...,...
29995,"Сэндвичи с лососем на гриле, рукколой…",0
29996,Маринованные креветки с луком и специями,0
29997,Заключение КТ: Что означает? Профилактический ...,1
29998,"Доброго времени суток. Хотела узнать, являются...",1


Сохраним датасеты для последующей работы:

In [31]:
data_0.to_csv(path_d + 'kook_data.csv', encoding='utf-8', index=False)
data_1.to_csv(path_d + 'qa_medical.csv', encoding='utf-8', index=False)
data_2.to_csv(path_d + 'bash_data.csv', encoding='utf-8', index=False)

In [32]:
data.to_csv(path_d + 'kook_med_bash_data.csv', encoding='utf-8', index=False)

In [33]:
data = pd.read_csv(path_d + 'kook_med_bash_data.csv')
data

,text,class
0,xxx: Я еще детям своим про крах Мелкософта рас...,2
1,Хумус из свеклы,0
2,Полезная пицца,0
3,"Кальмары, тушенные в сметане",0
4,Нежное песочное печенье,0
...,...,...
29995,"Сэндвичи с лососем на гриле, рукколой…",0
29996,Маринованные креветки с луком и специями,0
29997,Заключение КТ: Что означает? Профилактический ...,1
29998,"Доброго времени суток. Хотела узнать, являются...",1


Разобьем датасет на тренировочную и валидационную выборку:

In [34]:
text_train, text_valid, y_train, y_valid = train_test_split(data['text'],
                                                            data['class'],
                                                            test_size=.2,
                                                            shuffle=True,
                                                            stratify=data['class'],
                                                            random_state=42)

Загрузим мультилингвальную модель для классификации с количеством классов, равным 3:

In [35]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=3)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Токенизируем датасет и создадим датасет для обучения и валидации:

In [36]:
tf_train = tokenizer(text_train.tolist(), padding="max_length", truncation=True, return_tensors='tf')

In [37]:
tf_valid = tokenizer(text_valid.tolist(), padding="max_length", truncation=True, return_tensors='tf')

In [38]:
num_classes = 3
tf_y_train = tf.keras.utils.to_categorical(y_train.tolist(), num_classes)
tf_y_valid = tf.keras.utils.to_categorical(y_valid.tolist(), num_classes)

In [39]:
train_features = {x: tf_train[x] for x in tokenizer.model_input_names}
train_tf_dataset = tf.data.Dataset.from_tensor_slices((train_features, tf_y_train))
train_tf_dataset = train_tf_dataset.shuffle(len(tf_train)).batch(4)

valid_features = {x: tf_valid[x] for x in tokenizer.model_input_names}
valid_tf_dataset = tf.data.Dataset.from_tensor_slices((valid_features, tf_y_valid))
valid_tf_dataset = valid_tf_dataset.batch(4)

Заморозим бертовый слой, оставив для обучения только слой классификации, и скомпилируем модель:

In [40]:
model.layers[0].trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=[tf.metrics.CategoricalAccuracy('accuracy')],
)

model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  177853440 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  2307      
                                                                 
Total params: 177855747 (678.47 MB)
Trainable params: 2307 (9.01 KB)
Non-trainable params: 177853440 (678.46 MB)
_________________________________________________________________


Обучим модель:

In [41]:
model.fit(train_tf_dataset, validation_data=valid_tf_dataset, epochs=2)

Epoch 1/2
6000/6000 [==============================] - 1533s 252ms/step - loss: 0.3142 - accuracy: 0.9003 - val_loss: 0.1432 - val_accuracy: 0.9563
Epoch 2/2
6000/6000 [==============================] - 1569s 262ms/step - loss: 0.1837 - accuracy: 0.9347 - val_loss: 0.1237 - val_accuracy: 0.9528


In [42]:
#model.fit(train_tf_dataset, validation_data=valid_tf_dataset, epochs=1)

Сохраним модель и токенайзер:

In [43]:
tokenizer.save_pretrained(path_m + 'tokenizer_intent_classifier/')
model.save_pretrained(path_m + 'model_intent_classifier/')

Функция обратного вызова для построения отчета классификации при валидации:

In [44]:
class Metrics(Callback):


    def on_test_end(self, epoch, logs=None):
        val_predict = model.predict(valid_tf_dataset).logits.argmax(axis=1)
        print('\n', classification_report(y_valid.astype('int32'), val_predict))

Построим отчет классификации на валидационном датасете:

In [45]:
model.evaluate(valid_tf_dataset, callbacks=[Metrics()])

1500/1500 [==============================] - 280s 185ms/step

               precision    recall  f1-score   support

           0       0.99      1.00      0.99      2000
           1       0.94      0.92      0.93      2000
           2       0.93      0.94      0.93      2000

    accuracy                           0.95      6000
   macro avg       0.95      0.95      0.95      6000
weighted avg       0.95      0.95      0.95      6000

1500/1500 [==============================] - 560s 374ms/step - loss: 0.1237 - accuracy: 0.9528


[0.12373140454292297, 0.952833354473114]

**Вывод по классификатору:**
Я ожидал, что модель будет сильно путать 1-й и 2-й классы (обращения к врачу и отзывы на лекарства), но модель с хорошей точностью разделяет все три интента.

In [46]:
tokenizer1 = AutoTokenizer.from_pretrained(path_m + 'tokenizer_intent_classifier/')
model1 = TFAutoModelForSequenceClassification.from_pretrained(path_m + 'model_intent_classifier/')

Some layers from the model checkpoint at /content/drive/MyDrive/Data Science_models/nlp/project/model_intent_classifier/ were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at /content/drive/MyDrive/Data Science_models/nlp/project/model_intent_classifier/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassifi

In [47]:
def intent_classifier(sentence):
    sent_token = tokenizer1(sentence, padding="max_length", truncation=True, return_tensors='tf')
    predict = model1(sent_token).logits.numpy()
    print(predict)
    return predict.argmax()

In [48]:
intent_classifier('У меня сильный насморк, подскажите, что делать?')

[[-9.3093815  3.5555205 -2.0675576]]


1

In [57]:
intent_classifier('давай обсудим мой новый пылесос)')

[[-2.582087  -0.8879461  0.2680775]]


2

In [61]:
intent_classifier('родственники опять привезли елку :-)')

[[-0.05987115 -3.0260358   1.3603104 ]]


2

In [51]:
intent_classifier('С фарингосептом стало полегче')

[[-1.7514719  0.6092647 -1.8262738]]


1

In [52]:
intent_classifier('готовим омлет')

[[ 4.353205  -1.6324949 -2.2184381]]


0

In [62]:
intent_classifier('придумай новый рецепт борща')

[[ 0.48023486 -0.66646165 -1.7463062 ]]


0

In [46]:
intent_classifier('Арбидол не помогает улучшить иммунитет')

[[-2.7781928  2.0433192 -2.3682628]]


1

Классификатор работает. Хуже всего определяет класс болталки, видимо, потому что взятые описания медицинских проблем по стилю речи походят на обычную беседу (жалобы другу). Отличительной чертой датасета болталки является наличие смайликов, поэтому классификатор переобучился еще и на смайлики в данном классе.  